In [3]:


import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np



In [11]:
url = 'https://raw.githubusercontent.com/vikashishere/YT-Capstone-Project/main/notebooks/IMDB.csv'
df = pd.read_csv(url)
df = df.sample(500)
df.head()

,review,sentiment
346,"I'm hearing rumors of an upcoming ""Leonard Nim...",negative
56,Oliver Stone is not one to shy away from a mov...,positive
881,"or: It's a bird ? It's a plane ? No, look... I...",negative
270,"Long, boring, blasphemous. Never have I been s...",negative
400,"I`ve seen this movie twice, both times on Cine...",positive


In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saini\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [16]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saini\AppData\Roaming\nltk_data...


True

In [12]:
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [17]:
df = normalize_text(df)
df.head()

,review,sentiment
346,hearing rumor upcoming leonard nimoy demonstra...,negative
56,oliver stone one shy away movie theme matter e...,positive
881,bird plane look disaster need look sky br br o...,negative
270,long boring blasphemous never glad see ending ...,negative
400,seen movie twice time cinemax first time unrat...,positive


In [18]:
df['sentiment'].value_counts()

sentiment
negative    270
positive    230
Name: count, dtype: int64

In [20]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})


In [22]:


df.isnull().sum()



review       0
sentiment    0
dtype: int64

In [23]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [24]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Importing the mlflow and dagshub

In [25]:


import dagshub

mlflow.set_tracking_uri('https://dagshub.com/sakshamsainisaini94/Capstone-project.mlflow')
dagshub.init(repo_owner='sakshamsainisaini94', repo_name='Capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")



❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\Capstone-project\venv\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=5e768cab-2a42-4424-9b51-ee236982dbcd&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1ff37ca368da83e3f989a04f4f438053f06ca036d3d63e15d0bce4e1ab43b93c




Accessing as sakshamsainisaini94

Initialized MLflow to track repo "sakshamsainisaini94/Capstone-project"

Repository sakshamsainisaini94/Capstone-project initialized!

2025/03/02 01:35:43 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/364166672db8423c8eb8b542efc07522', creation_time=1740859543636, experiment_id='0', last_update_time=1740859543636, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [26]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-03-02 01:37:58,915 - INFO - Starting MLflow run...
2025-03-02 01:38:00,040 - INFO - Logging preprocessing parameters...
2025-03-02 01:38:03,285 - INFO - Initializing Logistic Regression model...
2025-03-02 01:38:03,286 - INFO - Fitting the model...
2025-03-02 01:38:03,341 - INFO - Model training complete.
2025-03-02 01:38:03,342 - INFO - Logging model parameters...
2025-03-02 01:38:03,778 - INFO - Making predictions...
2025-03-02 01:38:03,781 - INFO - Calculating evaluation metrics...
2025-03-02 01:38:03,796 - INFO - Logging evaluation metrics...
2025-03-02 01:38:05,444 - INFO - Saving and logging the model...
2025/03/02 01:38:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-02 01:38:19,404 - INFO - Model training and logging completed in 19.36 seconds.
2025-03-02 01:38:19,406 - INFO - Accuracy: 0.67
2025-03-02 01:38:19,407 - INFO - Precision: 0

🏃 View run ambitious-fawn-664 at: https://dagshub.com/sakshamsainisaini94/Capstone-project.mlflow/#/experiments/0/runs/f43de7652c764d21ac2989ac5fe1d6c0
🧪 View experiment at: https://dagshub.com/sakshamsainisaini94/Capstone-project.mlflow/#/experiments/0
